In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2019/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [5]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [6]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [7]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,15,303,"1,540",5.1,74,9,16,102.7,5,3,73
1,RB,16,298,"1,494",5.0,88,11,8,93.4,2,2,62
2,RB,16,287,"1,387",4.8,84,6,15,86.7,1,0,56
3,RB,16,301,"1,357",4.5,33,4,12,84.8,3,2,78
4,RB,15,278,"1,230",4.4,59,8,7,82.0,6,3,75
...,...,...,...,...,...,...,...,...,...,...,...,...
330,QB,3,5,-7,-1.4,0,0,0,-2.3,0,0,0
331,P,16,2,-8,-4.0,0,0,0,-0.5,3,2,0
332,QB,3,2,-9,-4.5,1,0,0,-3.0,0,0,0
333,WR,16,1,-9,-9.0,-9,0,0,-0.6,0,0,0


In [8]:

total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2019
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Derrick Henry,TEN,RB,15,303,"1,540",5.1,74,9,16,102.7,5,3,73,2019
1,2,Nick Chubb,CLE,RB,16,298,"1,494",5.0,88,11,8,93.4,2,2,62,2019
2,3,Christian McCaffrey,CAR,RB,16,287,"1,387",4.8,84,6,15,86.7,1,0,56,2019
3,4,Ezekiel Elliott,DAL,RB,16,301,"1,357",4.5,33,4,12,84.8,3,2,78,2019
4,5,Chris Carson,SEA,RB,15,278,"1,230",4.4,59,8,7,82.0,6,3,75,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,330,Tim Boyle,GB,QB,3,5,-7,-1.4,0,0,0,-2.3,0,0,0,2019
331,332,Jordan Berry,PIT,P,16,2,-8,-4.0,0,0,0,-0.5,3,2,0,2019
332,333,Blake Bortles,LAR,QB,3,2,-9,-4.5,1,0,0,-3.0,0,0,0,2019
333,333,Michael Thomas,NO,WR,16,1,-9,-9.0,-9,0,0,-0.6,0,0,0,2019


In [9]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Derrick Henry,TEN,RB,15,303,"1,540",5.1,74,9,16,102.7,5,3,73,2019
1,2,Nick Chubb,CLE,RB,16,298,"1,494",5.0,88,11,8,93.4,2,2,62,2019
2,3,Christian McCaffrey,CAR,RB,16,287,"1,387",4.8,84,6,15,86.7,1,0,56,2019
3,4,Ezekiel Elliott,DAL,RB,16,301,"1,357",4.5,33,4,12,84.8,3,2,78,2019
4,5,Chris Carson,SEA,RB,15,278,"1,230",4.4,59,8,7,82.0,6,3,75,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,330,Tim Boyle,GB,QB,3,5,-7,-1.4,0,0,0,-2.3,0,0,0,2019
331,332,Jordan Berry,PIT,P,16,2,-8,-4.0,0,0,0,-0.5,3,2,0,2019
332,333,Blake Bortles,LAR,QB,3,2,-9,-4.5,1,0,0,-3.0,0,0,0,2019
333,333,Michael Thomas,NO,WR,16,1,-9,-9.0,-9,0,0,-0.6,0,0,0,2019


In [10]:
total_stats.to_csv("Rushing_2019.csv", index = False)
print("Data saved to Rushing_2019.csv")

Data saved to Rushing_2019.csv
